# Coronavirus Latest
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide_colab_badge: true
- comments: false
- permalink:/covid-overview/


In [0]:
#hide
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML

base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'

# confirmed, deaths, recovered
def get_time_series(type):
  _url = f'{base_url}/csse_covid_19_time_series/time_series_covid19_{type}_global.csv'
  _dff = pd.read_csv(_url)
  _cols = _dff.columns[~_dff.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
  _dff = (_dff.groupby('Country/Region')[_cols].sum().stack().reset_index(name = 'Cases')
        .rename(columns = {'level_1': 'Date', 'Country/Region': 'Country'}))
  _dff['Date'] = pd.to_datetime(_dff['Date'], format='%m/%d/%y')
  return _dff

df_confirmed = get_time_series('confirmed')
df_recovered = get_time_series('recovered')
df_deaths = get_time_series('deaths')
date_latest = df_confirmed['Date'].max()

df_last = lambda name, _df, n: _df[_df['Date'].isin(_df['Date'].tail(n))].set_index(['Country', 'Date']).rename(columns={'Cases': name})
df_summary = lambda last_n: pd.concat([df_last('Confirmed', df_confirmed, last_n), df_last('Deaths', df_deaths, last_n), df_last('Recovered', df_recovered, last_n)], axis=1)
df_global = df_summary(2).groupby(['Date']).sum().reset_index()
s_date_latest = date_latest.strftime("%B %d, %Y")
n_confirmed = df_global.loc[1]['Confirmed']
n_deaths = df_global.loc[1]['Deaths']
n_recovered = df_global.loc[1]['Recovered']

df_global_diff = df_global.diff()
n_confirmed_diff = df_global_diff.loc[1]['Confirmed']
n_deaths_diff = df_global_diff.loc[1]['Deaths']
n_recovered_diff = df_global_diff.loc[1]['Recovered']
countries = df_confirmed['Country'].unique()

## Overview

In [2]:
#hide_input
HTML(
    f'<div style="font-size: 1.0em;">'
    f'<div> \
        <span style="font-size: 2.0em;">{n_confirmed:,}</span><br/>\
        Total cases reported worldwide, {s_date_latest} \
      </div>\
      <br/>'
    f'<div><span style="font-size: 2.0em;">{n_confirmed_diff:,.0f}</span><br/>New cases reported since last day</div><br/>'
    f'<div><span style="font-size: 2.0em;">{n_recovered:,}</span> +{n_recovered_diff:,.0f} ({n_recovered/n_confirmed*100:.2f}%)<br/>Total recovered reported worldwide</div><br/>'
    f'<div><span style="font-size: 2.0em;">{n_deaths:,}</span> +{n_deaths_diff:,.0f} ({n_deaths/n_confirmed*100:.2f}%)<br/>Total deaths reported worldwide</div><br/>'
    f'<div><span style="font-size: 2.0em;">{len(countries) - 1}</span><br/>Total number of countries tracked</div><br/>'
    f'</div>'
)

### Confirmed Cases

In [3]:
#hide_input
_df = df_confirmed.groupby(['Date']).sum().unstack().reset_index().drop(columns=['level_0']).rename(columns={0: 'Cummulative Confirmed'})
_df['New Confirm'] = _df[['Cummulative Confirmed']].diff()

_df_line = alt.Chart(_df).mark_line(color='#D32F2F').encode(
    alt.X('Date:T'),
    alt.Y('Cummulative Confirmed:Q', axis=alt.Axis(titleColor='#D32F2F'), scale=alt.Scale(type='log')),
    tooltip=list(_df)
)

_df_new_bar = alt.Chart(_df).mark_bar(opacity=0.3, color='#303F9F').encode(
    alt.X('Date:T'),
    alt.Y('New Confirm:Q', axis=alt.Axis(titleColor='#303F9F'), scale=alt.Scale(type='log')),
    tooltip=list(_df)
)


alt.layer(_df_line, _df_new_bar).resolve_scale(
    y='independent'
).properties(
    title = 'Confirmed Cases across the world',
    width = 600
)

alt.LayerChart(...)

In [0]:
#hide
def df_summary(_df):
  _summary = _df.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).tail(2).reset_index(drop=True).transpose()
  _summary['New'] = _summary.diff(axis=1)[1]
  _df_new_mean =_df.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).diff().rolling(window=7).mean()[-1:] \
                        .reset_index(drop=True).transpose().round(0).sort_values(by=0,ascending=False).rename(columns={0: 'New_Mean_7D'})                        
  _df_pct_chg = _df.groupby(['Date', 'Country']).sum().unstack().xs('Cases', axis=1).pct_change().rolling(window=7).mean()[-1:] \
                        .reset_index(drop=True).transpose().sort_values(by=0,ascending=False).rename(columns={0: 'Change_Mean_7D'})
  #df_confirmed_pct_mean_latest = df_confirmed_pct_mean
  _summary = pd.merge(_summary, _df_new_mean, on = 'Country')

  _summary['Change'] = _summary.pct_change(axis=1)[1]
  _summary = pd.merge(_summary, _df_pct_chg, on = 'Country')
  _summary['Trending'] = _summary['Change'] - _summary['Change_Mean_7D']
  _summary['Weight'] = _summary['New'] / _summary['New'].sum()
  _summary = _summary.drop(columns=0).rename(columns={1: 'Total'}).sort_values(by=['New', 'Total', 'Trending'], ascending=False).round(4)
  return _summary

df_summary_confirmed = df_summary(df_confirmed)

In [5]:
#hide_input
_data = df_summary_confirmed.reset_index().head(50)
_new_bar = alt.Chart().mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('New:Q', scale=alt.Scale(type='log')),
    tooltip = list(df_summary_confirmed)
)

_mean_bar = alt.Chart().mark_tick(color='red').encode(
    alt.X('Country:N'),# axis=alt.Axis(labels=False, ticks=False)),
    alt.Y('New_Mean_7D:Q', scale=alt.Scale(type='log')),
    tooltip = list(df_summary_confirmed)
)


_new_text = _new_bar.mark_text(
    angle=270,
    align='left',
    baseline='middle',    
    dy=3,
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='New:Q'
)

alt.layer(_new_bar + _new_text, data=_data).properties(
  title = f'New Confirmed Cases On {s_date_latest}',
  width= 600
)#.resolve_scale(x='independent')

alt.LayerChart(...)

In [6]:
#hide
df_summary_confirmed.sort_values(by='New', ascending=False).head(50)

,Total,New,New_Mean_7D,Change,Change_Mean_7D,Trending,Weight
Country,,,,,,,
US,839675,27639.0,29046.0,0.0340,0.0404,-0.0064,0.3729
Russia,57999,5236.0,4787.0,0.0992,0.1313,-0.0321,0.0706
United Kingdom,134638,4466.0,5022.0,0.0343,0.0442,-0.0099,0.0603
Spain,208389,4211.0,4392.0,0.0206,0.0231,-0.0025,0.0568
Italy,187327,3370.0,3167.0,0.0183,0.0182,0.0002,0.0455
Turkey,98674,3083.0,4183.0,0.0323,0.0516,-0.0194,0.0416
Brazil,45757,2678.0,2491.0,0.0622,0.0711,-0.0089,0.0361
Germany,150648,2357.0,2271.0,0.0159,0.0161,-0.0002,0.0318
Canada,41648,2247.0,1920.0,0.0570,0.0574,-0.0003,0.0303


In [7]:
#hide_input
_data = df_summary_confirmed[df_summary_confirmed['Total'] > 1000].reset_index()

_new_bar = alt.Chart().mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('Change:Q', axis=alt.Axis(format='%')),
    tooltip = list(df_summary_confirmed)
)

_mean_bar = alt.Chart().mark_tick(color='red').encode(
    alt.X('Country:N'),# axis=alt.Axis(labels=False, ticks=False)),
    alt.Y('Change_Mean_7D:Q',axis=alt.Axis(format='%')),
    tooltip = list(df_summary_confirmed)
)

_new_text = _new_bar.mark_text(
    size=10,
    angle=270,
    align='left',
    baseline='middle', 
    dy=3,
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Change:Q'
)

alt.layer(_new_bar + _new_text, data=_data).properties(
  title = f'Change of New Confirmed Cases (and 7 Days Average) On {s_date_latest}',
  width= 600
)#.resolve_scale(x='independent')

#alt.layer(_mean_bar, data=_data)

alt.LayerChart(...)

## Weight of Cases by Countries

In [8]:
#hide_input
df_confirmed_new = df_confirmed.set_index(['Date', 'Country']).unstack().diff().stack().reset_index()
df_confirmed_new_pct = df_confirmed_new.set_index(['Date', 'Country']).rename(columns={'Cases': 'New Confirm Weight'}).unstack().apply(lambda x: x/sum(x), axis=1).round(3).stack().reset_index()
selection = alt.selection_multi(fields=['Country'], bind='legend')
df_confirmed_new_pct = df_confirmed_new_pct[df_confirmed_new_pct['New Confirm Weight'] > 0.03]

_weight_bar = alt.Chart(df_confirmed_new_pct).mark_bar().encode(
    x='Date:T', 
    y=alt.Y('New Confirm Weight:Q', axis=alt.Axis(format='%'), title='Daily Confirm Weight'), 
    color='Country',
    tooltip=list(df_confirmed_new_pct),
    opacity=alt.condition(selection,alt.value(1), alt.value(0.2))
).add_selection(selection)
_weight_bar.properties(
    title = 'Weight of New Cases by Country (Country with weight more than 3%)',
    width = 600
)

alt.Chart(...)

In [9]:
#hide_input
df_summary_deaths = df_summary(df_deaths)
df_deaths_major = df_summary_deaths[(df_summary_deaths['New'] > 10) & (df_summary_deaths['Total'] > 100)].reset_index()

df_deaths_major.sort_values(by='New', ascending=False)


,Country,Total,New,New_Mean_7D,Change,Change_Mean_7D,Trending,Weight
0,US,46583,2139.0,2606.0,0.0481,0.0744,-0.0263,0.3320
1,United Kingdom,18151,773.0,751.0,0.0445,0.0501,-0.0057,0.1200
2,France,21373,544.0,598.0,0.0261,0.0317,-0.0055,0.0844
3,Italy,25085,437.0,491.0,0.0177,0.0213,-0.0036,0.0678
4,Spain,21717,435.0,430.0,0.0204,0.0216,-0.0011,0.0675
5,Belgium,6262,264.0,260.0,0.0440,0.0506,-0.0066,0.0410
6,Germany,5279,246.0,211.0,0.0489,0.0481,0.0008,0.0382
7,Sweden,1937,172.0,105.0,0.0975,0.0710,0.0264,0.0267
8,Canada,2075,167.0,153.0,0.0875,0.1106,-0.0231,0.0259
9,Brazil,2906,165.0,167.0,0.0602,0.0767,-0.0165,0.0256


In [10]:
#hide_input
alt.Chart(df_deaths_major).mark_bar().encode(
    alt.X('Country:N', sort='-y'),
    alt.Y('Change_Mean_7D:Q', axis=alt.Axis(format='%'), title='Mean Change'),
    tooltip=list(df_deaths_major)
).properties(
    width=600,
    title='Percentage Change (Mean of 7 Days) of Deaths Case'
)

alt.Chart(...)

In [11]:
#hide
df_confirmed_deaths = pd.merge(df_summary_confirmed.reset_index()[['Country', 'Total']].rename(columns={'Total': 'Confirmed'}), \
df_summary_deaths.reset_index()[['Country', 'Total']].rename(columns={'Total': 'Deaths'}), 
on='Country')

df_confirmed_deaths['Deaths Rate'] = df_confirmed_deaths['Deaths'] / df_confirmed_deaths['Confirmed']
df_confirmed_deaths[df_confirmed_deaths['Confirmed'] > 1000].sort_values(by='Deaths Rate', ascending=False).reset_index(drop=True).head(50)


,Country,Confirmed,Deaths,Deaths Rate
0,Belgium,41889,6262,0.149490
1,Algeria,2910,402,0.138144
2,France,157125,21373,0.136025
3,United Kingdom,134638,18151,0.134813
4,Italy,187327,25085,0.133910
5,Sweden,16004,1937,0.121032
6,Netherlands,35032,4068,0.116122
7,Spain,208389,21717,0.104214
8,Hungary,2168,225,0.103782
9,Mexico,9501,857,0.090201


## Cases in US

In [12]:
#hide 
def get_country(code):
  _df = [
          df_confirmed[df_confirmed['Country'] == code][['Date', 'Cases']].set_index('Date'), 
          df_deaths[df_deaths['Country'] == code][['Date', 'Cases']].set_index('Date').rename(columns={'Cases': 'Deaths'}),
          df_recovered[df_recovered['Country'] == code][['Date', 'Cases']].set_index('Date').rename(columns={'Cases': 'Recovered'})
        ]
  _df = pd.concat(_df, axis=1, join='inner')
  _df['Active'] = _df['Cases'] - _df['Deaths'] - _df['Recovered']
  _df = pd.concat( [_df, _df.diff().rename(columns={'Cases': 'New Cases', 'Deaths': 'New Deaths', 'Recovered': 'New Recovered', 'Active': 'New Active'})], axis=1, join='inner')
  _df['Deaths_Rate'] = _df['Deaths'] / _df['Cases']
  _df['Recovered_Rate'] = _df['Recovered'] / _df['Cases']
  return _df.reset_index()

country = 'US'
df_country_cases = get_country(country)
df_country_cases

,Date,Cases,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered,New Active,Deaths_Rate,Recovered_Rate
0,2020-01-22,1,0,0,1,NaN,NaN,NaN,NaN,0.000000,0.000000
1,2020-01-23,1,0,0,1,0.0,0.0,0.0,0.0,0.000000,0.000000
2,2020-01-24,2,0,0,2,1.0,0.0,0.0,1.0,0.000000,0.000000
3,2020-01-25,2,0,0,2,0.0,0.0,0.0,0.0,0.000000,0.000000
4,2020-01-26,5,0,0,5,3.0,0.0,0.0,3.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
87,2020-04-18,732197,38664,64840,628693,32491.0,1877.0,6295.0,24319.0,0.052805,0.088555
88,2020-04-19,758809,40661,70337,647811,26612.0,1997.0,5497.0,19118.0,0.053585,0.092694
89,2020-04-20,784326,42094,72329,669903,25517.0,1433.0,1992.0,22092.0,0.053669,0.092218
90,2020-04-21,812036,44444,75204,692388,27710.0,2350.0,2875.0,22485.0,0.054732,0.092612


In [13]:
#hide_input
_new_bar = alt.Chart(df_country_cases).mark_bar().encode(
    alt.X('Date:T'),
    alt.Y('New Cases:Q'),
    tooltip = list(df_country_cases)
)

_total_line = alt.Chart(df_country_cases[df_country_cases['Cases'] > 10]).mark_line(color='#57A44C').encode(
    alt.X('Date:T'),
    alt.Y('Cases:Q', axis=alt.Axis(titleColor='#57A44C'), scale=alt.Scale(type='log')),
    tooltip = list(df_country_cases)
)

alt.layer(_total_line, _new_bar).properties(
    width = 600,
    title = f'Number of Total Cases (Daily New Cases) for {country}'
).resolve_scale(y='independent')

alt.LayerChart(...)

In [14]:
#hide_input
_new_bar = alt.Chart(df_country_cases).mark_bar().encode(
    alt.X('Date:T'),
    alt.Y('New Deaths:Q'),
    tooltip = list(df_country_cases)
)

_total_line = alt.Chart(df_country_cases[df_country_cases['Deaths'] > 10]).mark_line(color='#57A44C').encode(
    alt.X('Date:T'),
    alt.Y('Deaths:Q', axis=alt.Axis(titleColor='#57A44C'), scale=alt.Scale(type='log')),
    tooltip = list(df_country_cases)
)

alt.layer(_total_line, _new_bar).properties(
    width = 600,
    title = f'Number of Total Deaths Cases (Daily Deaths Cases) for {country}'
).resolve_scale(y='independent')

alt.LayerChart(...)

In [21]:
#hide_input
_data = df_country_cases[(df_country_cases['New Cases']) > 0 & (df_country_cases['Cases'] > 100)]
alt.Chart(_data).mark_line().encode(
    alt.X('Cases:Q', scale=alt.Scale(type='log')),
    alt.Y('New Cases:Q', scale=alt.Scale(type='log', domain=[0.1, 100000]))
).properties(width = 600)

alt.Chart(...)

In [15]:
#hide_input
_rate1_line = alt.Chart(df_country_cases).mark_line(color='red').encode(
    alt.X('Date:T'),
    alt.Y('Deaths_Rate:Q', axis=alt.Axis(format='%', titleColor='red')),
    tooltip = list(df_country_cases)
) 

_rate2_line = alt.Chart(df_country_cases).mark_line(color='green').encode(
    alt.X('Date:T'),
    alt.Y('Recovered_Rate:Q', axis=alt.Axis(format='%', titleColor='green')),
    tooltip = list(df_country_cases)
)

(_rate1_line + _rate2_line).properties(
    width = 600,
    title = f'Rate of Deaths and Recovered for {country}'
).resolve_scale(y='independent')

alt.LayerChart(...)

## Deaths Cases Global


In [16]:
#hide_input
countries = ['China', 'Korea, South', 'Italy', 'Spain', 'US', 'Belgium']
_df = df_deaths.pivot(index='Country', columns='Date').diff(axis=1)['Cases'].stack().reset_index().rename(columns={0: 'New Deaths'})
_data = _df[_df['Country'].isin(countries)]

selection = alt.selection_multi(fields=['Country'], bind='legend')
alt.Chart(_data).mark_bar(opacity = 0.8).encode(
    alt.X('Date:T'),
    alt.Y('New Deaths:Q'),# scale=alt.Scale(type='log')),
    color='Country:N',
    tooltip = list(_data),
    opacity=alt.condition(selection,alt.value(1), alt.value(0.2))
).add_selection(selection).properties(
    width = 600
)

alt.Chart(...)

In [17]:
#hide_input
_df_objs = [
              df_deaths[df_deaths['Country'].isin(countries)].set_index(['Country', 'Date']).rename(columns={'Cases': 'Deaths'}),
              df_confirmed[df_confirmed['Country'].isin(countries)].set_index(['Country', 'Date']).rename(columns={'Cases': 'Confirmed'}) 
           ]
_df_countries = pd.concat(_df_objs, join='inner', axis=1).reset_index()
_df_countries['Deaths Rate'] = _df_countries['Deaths'] / _df_countries['Confirmed']
alt.Chart(_df_countries.fillna(0)).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Deaths Rate:Q', axis=alt.Axis(format='%')),
    color='Country',
    tooltip=list(_df_countries),
    opacity=alt.condition(selection,alt.value(1), alt.value(0.3))
).add_selection(selection).properties(
    width = 600
)

alt.Chart(...)